In [1]:
from pyspark.sql import SparkSession
    #.master("local[*]") \
spark = SparkSession.builder \
    .appName("IcebergAndBigQueryIntegration") \
    .master("yarn") \
    .config("spark.submit.deployMode", "client") \
    .config("spark.jars", "gs://tpch-source/jars/iceberg-spark-runtime-3.5_2.12-1.7.1.jar") \
    .config("spark.executor.instances", "1") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "4176m") \
    .config("spark.driver.memory", "2048m") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hadoop") \
    .config("spark.sql.catalog.iceberg.warehouse", "gs://tpch-source/iceberg_warehouse") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

# Verify Spark session
print(spark.sparkContext.getConf().getAll())


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/08 05:55:35 INFO SparkEnv: Registering MapOutputTracker
25/02/08 05:55:35 INFO SparkEnv: Registering BlockManagerMaster
25/02/08 05:55:35 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/02/08 05:55:35 INFO SparkEnv: Registering OutputCommitCoordinator


[('spark.sql.catalog.iceberg.warehouse', 'gs://tpch-source/iceberg_warehouse'), ('spark.dynamicAllocation.minExecutors', '1'), ('spark.eventLog.enabled', 'true'), ('spark.sql.shuffle.partitions', '1000'), ('spark.dataproc.sql.joinConditionReorder.enabled', 'true'), ('spark.app.initial.jar.urls', 'gs://tpch-source/jars/iceberg-spark-runtime-3.5_2.12-1.7.1.jar'), ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES', 'http://cluster-be89-m.local.:8088/proxy/application_1738975127572_0002'), ('spark.ui.proxyBase', '/proxy/application_1738975127572_0002'), ('spark.dataproc.sql.local.rank.pushdown.enabled', 'true'), ('spark.yarn.unmanagedAM.enabled', 'true'), ('spark.executorEnv.PYTHONPATH', '{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.9.7-src.zip'), ('spark.ui.filters', 'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'), ('spark.yarn.historyServer.address', 'cluster-be89-m:18080'), ('spark.sql.optimizer.runtime.bloomFilter.join.pattern.enabled',

In [2]:
# Read from BigQuery table
df = spark.read \
    .format("bigquery") \
    .option("table", "iceberg-feb-20205.movies.moviest") \
    .load()

# Show first 10 rows
df.show(10)

+-------+--------------------+------------------+
|movieId|               title|            genres|
+-------+--------------------+------------------+
| 114335|   La cravate (1957)|(no genres listed)|
| 122888|      Ben-hur (2016)|(no genres listed)|
| 122896|Pirates of the Ca...|(no genres listed)|
| 129250|   Superfast! (2015)|(no genres listed)|
| 132084| Let It Be Me (1995)|(no genres listed)|
| 134861|Trevor Noah: Afri...|(no genres listed)|
| 141131|    Guardians (2016)|(no genres listed)|
| 141866|   Green Room (2015)|(no genres listed)|
| 142456|The Brand New Tes...|(no genres listed)|
| 143410|          Hyena Road|(no genres listed)|
+-------+--------------------+------------------+
only showing top 10 rows



In [2]:
# Replace this with your actual DataFrame
df = spark.createDataFrame([
    (1, "Movie A", 2023),
    (2, "Movie B", 2024),
    (3, "Movie C", 2022)
], ["movie_id", "title", "year"])

# Write to BigQuery
df.write \
    .format("bigquery") \
    .option("table", "iceberg-feb-20205.movies.moviest2") \
    .option("temporaryGcsBucket", "gs://tpch-source/gks/temp/bg/") \
    .mode("overwrite") \
    .save()


In [8]:
df = spark.format("bigquery").table("`iceberg-feb-20205.movies.moviest2`")
df.show(5)

AttributeError: 'SparkSession' object has no attribute 'format'

In [3]:
# Read from BigQuery table
df = spark.read \
    .format("bigquery") \
    .option("table", "iceberg-feb-20205.movies.moviest2") \
    .load()

# Show first 10 rows
df.show(10)

+--------+-------+----+
|movie_id|  title|year|
+--------+-------+----+
|       1|Movie A|2023|
|       2|Movie B|2024|
|       3|Movie C|2022|
+--------+-------+----+



In [5]:
spark.stop()